Script: to concat files and assign top EF
--

In [1]:
import pandas as pd
import os
import seaborn as sns

Function to filter our background exons
--

In [66]:
def filter_and_save_exons(type_ss):
    all_EFs_df=pd.DataFrame()
    directory_path = '/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/00_data/emp_bayes_output/'+type_ss+'_outputs/assigned_EFs/'
    files=os.listdir(directory_path)

    # Loop through each file in the directory
    for file_name in files:
        # Check if the entry is a file (not a directory)
        if file_name!= '.ipynb_checkpoints':
    
            df = pd.read_csv(directory_path+file_name)
    
            df['tissue']=file_name.split('_EFs')[0]
    
            all_EFs_df=pd.concat([all_EFs_df, df])
            # Add your code here to do something with the file
    
    #check for convergence
    all_EFs_df = all_EFs_df[all_EFs_df['function_output_emp_bayes'].apply(lambda x: x.split('success: ')[1].split('\n')[0])=='True']
    
    #filter background
    all_EFs_background_SE = all_EFs_df[~all_EFs_df.passed_min_threshold_in_tissue]
    
    all_EFs_above_5_percent_PSI = all_EFs_df[all_EFs_df.passed_min_threshold_in_tissue]
    
    EF_5_bin = pd.cut(all_EFs_above_5_percent_PSI.EF_5, [0,0.1,0.3,0.5,0.7,0.9,1.0])
    
    all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(EF_5_bin=EF_5_bin)
    

    if type_ss=='alt_ss':
        intron_1=all_EFs_above_5_percent_PSI.intron_1.apply(lambda x: x.split(':clu')[0])

        intron_2=all_EFs_above_5_percent_PSI.intron_2.apply(lambda x: x.split(':clu')[0])
        
        cluster_name_no_ID=intron_1+'_'+intron_2
        
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(cluster_name_no_ID=cluster_name_no_ID)
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(splicing_event='alt_splice_site')
        
        

    elif type_ss=='se':

            
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_1_unskipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x:  x.split('_chr')[0]))
    ########
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_2_unskipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[2]+':'+x.split(':')[3].split('_chr')[0]))
                                                                                                                                  
    
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_skipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1]))
    

        exclusion=all_EFs_above_5_percent_PSI[all_EFs_above_5_percent_PSI.minority_is_exclusion_event]
        
        # For 'exclusion' DataFrame
        intron_1_exclusion = exclusion.cluster_name.apply(lambda x: x.split('_chr')[0])
        exclusion = exclusion.assign(intron_1=intron_1_exclusion)
        
        intron_2_exclusion = exclusion.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1])


        exclusion = exclusion.assign(intron_2=intron_2_exclusion)
        
        # For 'inclusion' DataFrame
        inclusion=all_EFs_above_5_percent_PSI[~all_EFs_above_5_percent_PSI.minority_is_exclusion_event]
        
        intron_1_inclusion = inclusion.cluster_name.apply(lambda x: x.split('_chr')[0])
        inclusion = inclusion.assign(intron_1=intron_1_inclusion)


        
        intron_2_inclusion = inclusion.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1])
        inclusion = inclusion.assign(intron_2=intron_2_inclusion)
        
        # Combining 'inclusion' and 'exclusion' DataFrames
        all_EFs_above_5_percent_PSI = pd.concat([inclusion, exclusion])
        
                
        all_EFs_above_5_percent_PSI=pd.concat([inclusion, exclusion])

        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(splicing_event='skipped_exon')
         
    top_EF_all_tissues = all_EFs_above_5_percent_PSI.sort_values(by='EF_5', ascending=False).drop_duplicates(subset='cluster_name_no_ID', keep='first')
    
  #  all_EFs_above_5_percent_PSI.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/all_EF_all_tissues_'+type_ss+'.csv.gz', 
                             # compression = 'gzip',
                             # index=True)
    
    #all_EFs_background.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/all_EF_background_set_'+type_ss+'.csv.gz', 
                             # compression = 'gzip',
                             # index=True)
    
   # top_EF_all_tissues.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/top_EF_all_tissues_'+type_ss+'.csv.gz', 
                             # compression = 'gzip',
                             # index=True)

    return all_EFs_above_5_percent_PSI, top_EF_all_tissues

        
   
        
                            
    

In [69]:
def filter_and_save_exons_new(type_ss):
    all_EFs_df=pd.DataFrame()
    directory_path = '/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/00_data/emp_bayes_output/'+type_ss+'_outputs/assigned_EFs/'
    files=os.listdir(directory_path)

    # Loop through each file in the directory
    for file_name in files:
        # Check if the entry is a file (not a directory)
        if file_name!= '.ipynb_checkpoints':
    
            df = pd.read_csv(directory_path+file_name)
    
            df['tissue']=file_name.split('_EFs')[0]
    
            all_EFs_df=pd.concat([all_EFs_df, df])
            # Add your code here to do something with the file
    
    #check for convergence
    all_EFs_df = all_EFs_df[all_EFs_df['function_output_emp_bayes'].apply(lambda x: x.split('success: ')[1].split('\n')[0])=='True']
    
    #filter background
    all_EFs_background_SE = all_EFs_df[~all_EFs_df.passed_min_threshold_in_tissue]
    
    all_EFs_above_5_percent_PSI = all_EFs_df[all_EFs_df.passed_min_threshold_in_tissue]
    
    EF_5_bin = pd.cut(all_EFs_above_5_percent_PSI.EF_5, [0,0.1,0.3,0.5,0.7,0.9,1.0])
    
    all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(EF_5_bin=EF_5_bin)
    

    if type_ss=='alt_ss':
        intron_1=all_EFs_above_5_percent_PSI.intron_1.apply(lambda x: x.split(':clu')[0])

        intron_2=all_EFs_above_5_percent_PSI.intron_2.apply(lambda x: x.split(':clu')[0])
        
        cluster_name_no_ID=intron_1+'_'+intron_2
        
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(cluster_name_no_ID=cluster_name_no_ID)
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(splicing_event='alt_splice_site')
        
        

    elif type_ss=='se':

            
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_1_unskipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x:  x.split('_chr')[0]))
    ########
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_2_unskipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[2]+':'+x.split(':')[3].split('_chr')[0]))
                                                                                                                                  
    
        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(intron_skipped=all_EFs_above_5_percent_PSI.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1]))
    

        exclusion=all_EFs_above_5_percent_PSI[all_EFs_above_5_percent_PSI.minority_is_exclusion_event]
        
        # For 'exclusion' DataFrame
        intron_1_exclusion = exclusion.cluster_name.apply(lambda x: x.split('_chr')[0])
        exclusion = exclusion.assign(intron_1=intron_1_exclusion)
        
        intron_2_exclusion = exclusion.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1])


        exclusion = exclusion.assign(intron_2=intron_2_exclusion)
        
        # For 'inclusion' DataFrame
        inclusion=all_EFs_above_5_percent_PSI[~all_EFs_above_5_percent_PSI.minority_is_exclusion_event]
        
        intron_1_inclusion = inclusion.cluster_name.apply(lambda x: x.split('_chr')[0])
        inclusion = inclusion.assign(intron_1=intron_1_inclusion)


        
        intron_2_inclusion = inclusion.cluster_name.apply(lambda x: 'chr'+x.split('_chr')[1])
        inclusion = inclusion.assign(intron_2=intron_2_inclusion)
        
        # Combining 'inclusion' and 'exclusion' DataFrames
        all_EFs_above_5_percent_PSI = pd.concat([inclusion, exclusion])
        
                
        all_EFs_above_5_percent_PSI=pd.concat([inclusion, exclusion])

        intron_1=all_EFs_above_5_percent_PSI.intron_1.apply(lambda x: x.split(':clu')[0])

        intron_2=all_EFs_above_5_percent_PSI.intron_2.apply(lambda x: x.split(':clu')[0])
        
        cluster_name_no_ID=intron_1+'_'+intron_2

        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(splicing_event='skipped_exon')

        all_EFs_above_5_percent_PSI=all_EFs_above_5_percent_PSI.assign(cluster_name_no_ID=cluster_name_no_ID)

        
         
    top_EF_all_tissues = all_EFs_above_5_percent_PSI.sort_values(by='EF_5', ascending=False).drop_duplicates(subset='cluster_name_no_ID', keep='first')
    
    all_EFs_above_5_percent_PSI.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/all_EF_all_tissues_'+type_ss+'.csv.gz', 
                              compression = 'gzip',
                              index=True)
    
    #all_EFs_background.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/all_EF_background_set_'+type_ss+'.csv.gz', 
                            #  compression = 'gzip',
                              #index=True)
    
    top_EF_all_tissues.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/top_EF_all_tissues_'+type_ss+'.csv.gz', 
                              compression = 'gzip',
                              index=True)

    return all_EFs_above_5_percent_PSI, top_EF_all_tissues

        
   
        
                            
    
                              

In [70]:
[all_EFs_above_5_percent_PSI_se, top_EF_all_tissues_se]= filter_and_save_exons_new('se')




In [77]:
intron_1=all_EFs_above_5_percent_PSI_alt_ss.intron_1.apply(lambda x: x.split(':clu')[0])

intron_2=all_EFs_above_5_percent_PSI_alt_ss.intron_2.apply(lambda x: x.split(':clu')[0])
        

In [82]:
example_of_no_cluster_IDs=intron_1+'_'+intron_2

In [84]:
example_of_no_cluster_IDs.value_counts()

chr15:88467782:88467911_chr15:88467782:88467908        49
chr7:6042990:6044562_chr7:6042993:6044562              49
chr21:31685174:31685398_chr21:31685240:31685398        49
chr14:105220127:105221648_chr14:105220130:105221648    49
chr14:105380154:105380954_chr14:105380154:105380957    49
                                                       ..
chr16:53492519:53492693_chr16:53692364:53696151         1
chr16:53435744:53439016_chr16:53453612:53453705         1
chr16:53307905:53308686_chr16:53434796:53439016         1
chr16:53304625:53306237_chr16:53307953:53308686         1
chrX:9760826:9765568_chrX:9760826:9786242               1
Name: count, Length: 190746, dtype: int64

In [80]:
intron_1.value_counts()

intron_1
chr12:123613306:123615185    49
chr10:77854358:77856730      49
chr19:15117972:15119166      49
chr17:10706550:10710834      49
chr16:53304625:53306237      49
                             ..
chr2:72575668:72576465        1
chr2:74465473:74472123        1
chr2:8047938:8087027          1
chr2:86889276:86893281        1
chrX:9760826:9765568          1
Name: count, Length: 138953, dtype: int64

In [78]:
intron_2.value_counts()

intron_2
chr12:121004492:121005003    49
chr4:48546271:48547584       49
chr15:43397321:43398041      49
chr11:11936510:11938257      49
chr8:81680783:81681498       49
                             ..
chr3:195710310:195710536      1
chr3:196055752:196055816      1
chr3:197635807:197649539      1
chr3:197944401:197944968      1
chrX:49163929:49164707        1
Name: count, Length: 125684, dtype: int64

In [75]:
top_EF_all_tissues_alt_ss.intron_1.value_counts()

intron_1
chr11:67284983:67285078:clu_6769      2
chr11:63270112:63297303:clu_5338      2
chr1:209760324:209762515:clu_45447    2
chr5:81086953:81092801:clu_31663      2
chr3:179149716:179198750:clu_37049    2
                                     ..
chr6:119817133:119843066:clu_38731    1
chr15:23979211:24003338:clu_15370     1
chr1:50441523:50451793:clu_52640      1
chr17:51192280:51192658:clu_13225     1
chr5:138169295:138170345:clu_35668    1
Name: count, Length: 190740, dtype: int64

In [71]:
[all_EFs_above_5_percent_PSI_alt_ss, top_EF_all_tissues_alt_ss]= filter_and_save_exons_new('alt_ss')



sum stats
--

In [73]:
print('top EF SE# = '+str(len(top_EF_all_tissues_se)))

print('all EF SE# = '+str(len(all_EFs_above_5_percent_PSI_se)))

print('top EF altss# = '+str(len(top_EF_all_tissues_alt_ss)))

print('all EF altss# = '+str(len(all_EFs_above_5_percent_PSI_alt_ss)))

top EF SE# = 58579
all EF SE# = 278601
top EF altss# = 190746
all EF altss# = 793676


export this filtered set by EF, to filter for protein coding and maxent and size of exon in second (02) notebook
--

In [37]:




#SE= top_EF_all_tissues_se[['cluster_name', 'EF_1', 'EF_5', 'EF_10', 'EF_20','splicing_event','alpha_prior','beta_prior', 'intron_1', 'intron_2']]

#alt_ss =top_EF_all_tissues_alt_ss[['cluster_name', 'EF_1', 'EF_5', 'EF_10', 'EF_20','splicing_event','alpha_prior','beta_prior', 'intron_1', 'intron_2']]

#top_EF_df=pd.concat([SE, alt_ss])

#top_EF_df.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/top_EF_all_events_summary_of_EFs.csv.gz', 
                         # compression = 'gzip',
                         #)



#alt_ss = all_EFs_above_5_percent_PSI_alt_ss=all_EFs_above_5_percent_PSI_alt_ss.assign(splicing_event='alt_ss')

#SEs = all_EFs_above_5_percent_PSI_se=all_EFs_above_5_percent_PSI_se.assign(splicing_event='skipped_exon')


#all_EF_df=pd.concat([SE, alt_ss])

#all_EF_df.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/all_EF_all_events_summary_of_EFs.csv.gz', 
                          #compression = 'gzip')s
                         



In [31]:
#top_EF_all_tissues_se.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/filtered_maxent_top_EF_all_tissues_skipped_exons.csv.gz', compression='gzip', index=False)

#top_EF_all_tissues_alt_ss.to_csv('/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/filtered_maxent_top_EF_all_tissues_alt_ss.csv.gz', compression='gzip', index=False)
data_directory='/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/01_data/new_results/'

In [32]:
top_EF_all_tissues_se.to_csv(data_directory+'top_EF_all_tissues_se.csv.gz',
                                             compression='gzip',
                                             index=False)

top_EF_all_tissues_alt_ss.to_csv(data_directory+'top_EF_all_tissues_alt_ss.csv.gz',
                                             compression='gzip',
                                             index=False)


all_EFs_above_5_percent_PSI_se.to_csv(data_directory+'all_EF_all_tissues_se.csv.gz',
                                             compression='gzip',
                                             index=False)

all_EFs_above_5_percent_PSI_alt_ss.to_csv(data_directory+'all_EF_all_tissues_alt_ss.csv.gz',
                                             compression='gzip',
                                             index=False)